## 📦 Importações e Configuração Inicial

Nesta seção, o código importa todas as bibliotecas necessárias para o funcionamento do agente CrewAI, ferramentas externas (como busca e leitura de CSV), além de utilitários como `datetime`, `os`, `dotenv` e módulos do LangChain.

Isso prepara o ambiente com tudo que será usado nas próximas etapas do projeto.


In [ ]:
import json
import os
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

## 🤖 Configuração do LLM e Ferramentas de Busca

Aqui carregamos as variáveis de ambiente com `load_dotenv()` para usar chaves de API e configuramos o modelo de linguagem `ChatOpenAI`, definindo o modelo, temperatura e limite de tokens.

Também inicializamos a ferramenta de busca `DuckDuckGoSearchResults`, que poderá ser usada pelo agente para obter informações atualizadas da internet.


In [ ]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-3.5-turbo-0125") # temperature=0)
search_tool = DuckDuckGoSearchResults()

## 🏠 Geração de Banco de Dados Fictício (CSV de Imóveis)

Este trecho de código gera um arquivo CSV com dados simulados de imóveis para uma imobiliária fictícia.

- **Endereços, tipos de imóvel e características** são aleatoriamente gerados.
- O arquivo é salvo em `files/imoveis.csv` com colunas como: `ID`, `Endereço`, `Preço`, `Quartos`, `Banheiros`, `Metragem` e `Tipo`.
- Isso permite simular um banco de dados que poderá ser consultado por agentes IA.

🔧 Útil para testes e desenvolvimento sem depender de dados reais.


## 🔍 Integração do Agente com o CSV (Base de Imóveis)

Nesta etapa, a ferramenta `CSVSearchTool` é usada para permitir que o agente IA consulte dados diretamente do arquivo `imoveis.csv`.

Com isso, o agente pode realizar buscas como:
- "Quais imóveis têm mais de 3 quartos?"
- "Liste casas abaixo de 500 mil reais"

📌 Isso transforma o CSV em uma **fonte de dados ativa** para tomada de decisão ou resposta a perguntas.


In [ ]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")

## 🧑‍💼 Definição do Agente: Corretor de Imóveis

Este agente representa um corretor especializado, com a missão de:
- **Entender as preferências do cliente**
- **Buscar imóveis compatíveis** no arquivo `imoveis.csv` (usando `CSVSearchTool`)

### ⚙️ Configurações do agente:
- **role:** Define o papel (Corretor de Imóveis)
- **goal:** Descreve o objetivo final do agente
- **backstory:** Dá contexto e especialização ao comportamento do agente
- **tools:** Integração com o CSV para fazer buscas nos imóveis
- **verbose:** Mostra o processo de raciocínio do agente passo a passo
- **max_iter:** Número máximo de iterações para alcançar o objetivo
- **allow_delegation:** `False` porque esse agente trabalha sozinho
- **memory:** `True`, o agente pode lembrar informações da conversa anterior

🧩 Este agente é uma peça essencial no sistema, atuando diretamente na simulação de atendimento imobiliário inteligente.


In [ ]:
# Corretor de Imóveis
corretor_imoveis = Agent(
    role="Corretor de Imóveis",
    goal="Obtenha as preferencias do cliente e busque imóveis compativeis no banco de dados.",
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas nas preferências do cliente.",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True,
)


## 🔎 Tarefa: Buscar Imóveis Conforme Preferências do Cliente

Esta tarefa define o que o agente "Corretor de Imóveis" deve executar. O foco está em localizar imóveis que:

- Se encaixem nas **preferências do cliente**
- Levem em conta **preço**, **localização** e **tipo de imóvel**

### 🧾 Detalhes da tarefa:
- **description:** Enunciado que orienta a missão do agente
- **expected_output:** Resultado esperado: uma **lista com detalhes relevantes dos imóveis encontrados**
- **agent:** O agente encarregado (neste caso, `corretor_imoveis`)
- **verbose:** Ativo para mostrar o raciocínio passo a passo durante a execução

🔧 Essa tarefa direciona a atuação do agente para um objetivo específico dentro da simulação de atendimento.


In [ ]:
# Tarefa de Buscar Imóveis
buscar_imoveis = Task(
    description="Busque imóveis que atendam às preferências do cliente, consideranbdo o preço, localização e tipo de imóvel.",
    expected_output="Lista de imoveis disponíveis com detalhes sobre localização, preço e características.",
    agent=corretor_imoveis,
    verbose=True,
)

## 📊 Ferramenta Personalizada: Analisador de Preço de Imóveis

Nesta seção é criada uma ferramenta customizada (`PrecoImoveisTool`) que permite ao agente consultar tendências de preço no mercado imobiliário.

### 🔧 Componentes:
- **Função `obter_preco_imoveis(cidade)`**:
  - Retorna uma tendência (`aumento`, `estável`, ou `queda`) e um percentual médio com base na cidade informada.
  - Se a cidade não for reconhecida, retorna os dados "gerais".

- **Classe `PrecoImoveisTool` (herda de `BaseTool`)**:
  - **name:** Nome da ferramenta exibido ao agente.
  - **description:** Explica o propósito da ferramenta.
  - **_run(cidade):** Método principal que executa a análise de tendência.

### 🎯 Finalidade:
Permitir que o agente IA:
- Ofereça insights atualizados de mercado
- Enriqueça a conversa com o cliente
- Fundamente melhor suas recomendações

📌 Ideal para contextos onde se deseja simular um corretor mais **estratégico e analítico**.

## 📈 Ferramenta Personalizada: Analisador de Preço de Imóveis

Esta ferramenta (`PrecoImoveisTool`) é uma classe baseada em `BaseTool` da CrewAI, que permite a um agente consultar a **tendência atual de preços** do mercado imobiliário em uma cidade específica.



In [ ]:
from crewai.tools import BaseTool

def obter_preco_imoveis(cidade: str = "geral"):
    preços= {
        "São Paulo": {"tendencia":"aumento", "percentual": 5.2},
        "Rio de Janeiro": {"tendencia":"estavel", "percentual": 0.0},
        "Belo Horizonte": {"tendencia":"queda", "percentual": -3.1},
        "Curitiba": {"tendencia":"queda", "percentual": -2.5},
        "geral": {"tendencia":"aumento", "percentual": 4.0},
    }
    return preços.get(cidade, preços["geral"])

class PrecoImoveisTool(BaseTool):
    name: str = "Analisador de Preço de Imóveis"
    description: str = "Retorna a tendencia de preço dos imóveis em uma cidade específica ou no geral."
    
    def _run(self, cidade: str) -> dict:
        """
        Executa a analise de preços imobiliarios e retorna a tendencia com base na cidade.
        """
        try:
            return obter_preco_imoveis(cidade)
        except Exception as e:
            return {"error": f"Erro ao obter tendencias de preços {str(e)}"}

## 📈 Agente: Analista de Mercado Imobiliário

Este agente é responsável por analisar **tendências e variações de preços** no setor imobiliário, com base em dados por região.

### ⚙️ Parâmetros configurados:
- **role:** Define o papel do agente (Analista de Mercado Imobiliário)
- **goal:** Objetivo principal é analisar preços por região
- **backstory:** Contextualiza o agente como um especialista em comportamento do mercado imobiliário
- **verbose:** Ativo para exibir os passos de raciocínio
- **max_iter:** Até 5 iterações para alcançar seu objetivo
- **allow_delegation:** `False`, o agente atua de forma independente
- **memory:** `True`, o agente pode lembrar informações durante a execução

### 📌 Observação:
- A ferramenta `PrecoImoveisTool` está comentada no momento.
- Para ativá-la, basta descomentar a linha `tools=[PrecoImoveisTool()]`.

🔍 Este agente é ideal para oferecer **contexto de mercado** ao cliente ou ao corretor, com foco estratégico.


In [ ]:
analista_mercado = Agent(
    role="Analista de Mercado Imobiliário",
    goal="Analisa as tendências de preços dos imóveis em diferentes regiões.",
    backstory="Especialista em análise de mercado, fornece insights sobre tendências e variações de preços.",
    verbose=True,
    max_iter=5,
    # tools=[PrecoImoveisTool()], usado na TASK
    allow_delegation=False,
    memory=True,
)

## 🧩 Seção 8 - Tarefa de Análise de Tendências do Mercado Imobiliário

### 🔍 `obter_tendencias` (Task)
Esta tarefa é atribuída ao **agente Analista de Mercado Imobiliário** e tem como objetivo analisar as tendências de preço dos imóveis com base em parâmetros específicos.


In [ ]:
obter_tendencias = Task(
    description="""
    Analise o historico de preços de imóveis na cidade {cidade} e forneça insights sobre valorização ou desvalorização.
    Considere o tipo de imóvel {tipo_imovel} e a faixa de preço {faixa_preco}.
    """,
    expected_output="Resumo da tendência dos preços no mercado imobiliário.",
    tools=[PrecoImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"],
    verbose=True,
)

## 📉 Analista de Notícias Imobiliárias

### 🧾 Descrição do Agente `analista_noticias`
- **Papel:** Analista de Notícias imobiliárias.
- **Objetivo:** Analisar as notícias mais recentes relacionadas ao mercado imobiliário para avaliar fatores externos que podem impactar o setor.
- **Histórico:** Especialista em análise de notícias, fornecendo insights sobre eventos que influenciam o mercado imobiliário.
- **Configurações:** `verbose=True` para detalhamento das operações, `max_iter=5` para limitar iterações e `memory=True` para manter contexto entre interações.

### 🔎Ferramenta de Busca `search_tool`
### 🛠️Aplicando a ferramenta de Busca `search_tool`
- Utiliza o backend `"news"` do DuckDuckGo para buscar as 5 notícias mais recentes relacionadas ao mercado imobiliário.

### 📝Tarefa `buscar_noticias`
- **Descrição:** Pesquisa notícias recentes sobre o mercado imobiliário (incluindo a data atual).
- **Resultado Esperado:** Resumo das principais notícias e tendências do mercado imobiliário.
- **Agente Responsável:** `analista_noticias`.
- **Ferramenta Usada:** `search_tool` para coleta de informações atuais da web.

### Observação
A combinação do agente com a ferramenta de busca permite que o sistema integre dados dinâmicos e atualizados, aumentando a relevância e precisão das análises.


In [ ]:
analista_noticias = Agent(
    role="Analista de Notícias imobiliarias",
    goal="Analisa as notícias mais recentes relacionadas ao mercado imobiliário., para avaliar fatores externos",
    backstory="Especialista em análise de notícias, fornece insights sobre eventos que podem impactar o mercado imobiliário.",
    verbose=True,
    max_iter=5,
    memory=True,
)

In [ ]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=5)

In [ ]:
search_tool

In [ ]:
buscar_noticias = Task(
    description=f"Pesquise noticias recentes sobre o mercado imomiliario. Data atual: {datetime.now()}.",
    expected_output="Resumo das principais noticias e tendências imobiliárias.",
    agent=analista_noticias,
    tool = [search_tool],
)

# 🧠 Agente Consultor Financeiro

### 🧾Descrição do Agente `consultor_financeiro`
- **🧑‍💼 Papel:** Consultor Financeiro
- **🎯 Objetivo:** Analisar a viabilidade financeira de um investimento imobiliário com base no perfil do cliente.
- **📚 Histórico:** Especialista em crédito imobiliário, com foco em auxiliar clientes na escolha das melhores opções de financiamento disponíveis no mercado.
  
### ⚙️ Configurações do Agente:
- **`verbose=True`**: Exibe detalhes do processo durante a execução, útil para depuração e acompanhamento.
- **`allow_delegation=False`**: Não permite que o agente delegue tarefas para outros agentes.
- **`max_iter=5`**: Limita o número máximo de iterações para alcançar a resposta.
- **`memory=True`**: Habilita memória para manter o contexto das interações anteriores com o agente.

### 📝 Finalidade
Este agente simula um consultor financeiro pessoal, que leva em consideração a situação econômica do cliente e os custos envolvidos no financiamento de imóveis, fornecendo uma análise fundamentada e orientações úteis.


In [ ]:
consultor_financeiro = Agent(
    role="Consultor Financeiro",
    goal="Analisa a viabilidade financeira de um investimento imobiliário, com base no perfil do cliente.",
    backstory="Especialista em credito imobiliário, ajuda clientes a escolherem as melhores opções de financiamento.",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [ ]:
calcular_financiamento = Task(
    description="Consultor Financeiro",
    expected_output="Analisa a viabilidade financeira de um investimento imobiliário, com base no perfil do cliente.",
    agent=consultor_financeiro,
)

## 🧠Agente Redator de Relatórios Imobiliários

### 🧑‍💻 Agente: `redator`
- **📝 Papel:** Redator de Relatórios Imobiliários
- **🎯 Objetivo:** Gerar relatórios completos, claros e persuasivos com base em análises de mercado e imóveis selecionados.
- **📚 Histórico:** Profissional de comunicação especializado em traduzir dados complexos do mercado imobiliário para uma linguagem acessível aos clientes.

### ⚙️ Configurações do Agente:
- **`verbose=True`**: Exibe detalhes do processo durante a execução.
- **`allow_delegation=False`**: O agente não pode delegar tarefas a outros agentes.
- **`max_iter=5`**: Número máximo de iterações permitido.
- **`memory=True`**: Mantém o contexto das interações anteriores, permitindo coerência nas respostas.

---

## 📝Tarefa: `gerar_relatorio`
- **📄 Descrição:** Gerar um relatório detalhado sobre o melhor imóvel encontrado, considerando preços, tendências de mercado e possibilidades de financiamento.
- **🎯 Saída Esperada:** Um relatório formatado contendo:
  - Resumo do mercado
  - Opções recomendadas
  - Justificativas da escolha

### 🔗 Contexto Utilizado:
- `buscar_imoveis`: Imóveis encontrados com base nas preferências do cliente.
- `obter_tendencias`: Análise de tendências do mercado imobiliário.
- `buscar_noticias`: Notícias recentes que impactam o setor.
- `calcular_financiamento`: Análise de viabilidade financeira para aquisição do imóvel.

### 🎯 Finalidade:
Transformar todos os dados e análises do processo em um relatório final bem estruturado e convincente, pronto para apresentação ao cliente.


In [ ]:
redator = Agent(
    role="Redator de Relatórios Imobiliários",
    goal="Gera um relatório completo e persuasivo com base nas análises de mercado e imóveis encontrados.",
    backstory="Especialista em comunicação, traduz dados complexos para clientes de forma clara e objetiva.",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

In [189]:
gerar_relatorio = Task(
    description="Gere um relatório detalhado sobre o melhor imóvel encontrado, considerando preços, tendências e financiamento.",
    expected_output="Relatório formatado com resumo do mercado, opções recomendads e justificativas da escolha, com tabelas para serem comparadas a outras opções.",
    agents=redator,
    context=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento],
)

# reunir agentes (CREW)

In [ ]:
crew = Crew(
    agents=[corretor_imoveis, analista_mercado, analista_noticias, consultor_financeiro, redator],
    tasks=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento, gerar_relatorio],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    max_iter=5,
    manager_llm=llm,
)

In [ ]:
result = crew.kickoff(inputs={"cidade": "São Paulo", "tipo_imovel": "apartamento", "faixa_preco": "500000-700000"})

In [ ]:
print(result.raw)

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(str(result)))